# Formatting data

In [2]:
suppressMessages(library(Seurat))
suppressMessages(library(PISCES))
suppressMessages(library(SeuratDisk))
suppressMessages(library(cowplot))
suppressMessages(library(patchwork))
suppressMessages(library(tidyverse))
theme_set(theme_cowplot())

In [3]:
# Read in the R object
dat <- readRDS("../../../data/mouse_adrenal/auxiliary_data/snrna/adrenal_Parse_10x_integrated.rds")

In [4]:
dat

An object of class Seurat 
82892 features across 79209 samples within 3 assays 
Active assay: SCT (32171 features, 0 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [29]:
raw.mat <- GetAssayData(dat, assay = "SCT", slot = "counts")

In [31]:
dim(raw.mat)

[1] 32171 79209

In [30]:
raw.mtx <- as.matrix(raw.mat)

ERROR: Error in asMethod(object): Cholmod error 'problem too large' at file ../Core/cholmod_dense.c, line 102


In [27]:
as.matrix(raw.mat[1:6, 1:6])

,AAACAGCCAATATACC.ENCLB069PRB,AAACAGCCATGGCCCA.ENCLB069PRB,AAACATGCAAAGCTCC.ENCLB069PRB,AAACATGCAACTAGGG.ENCLB069PRB,AAACATGCAGCAATAA.ENCLB069PRB,AAACCAACAACAGCCT.ENCLB069PRB
Xkr4,0,0,0,0,0,1
Gm18956,0,0,0,0,0,0
Gm1992,0,0,0,0,0,0
Gm7341,0,0,0,0,0,0
Gm19938,0,0,0,0,0,0
Gm37381,0,0,0,0,0,0


In [21]:
#' Performas a CPM normalization on the given data. 
#' 
#' @param dat.mat Matrix of gene expression data (genes X samples).
#' @param l2 Optional log2 normalization switch. Default of False.
#' @return Returns CPM normalized matrix
CPMTransform <- function(dat.mat, l2 = FALSE) {
  cpm.mat <- t(t(dat.mat) / (colSums(dat.mat) / 1e6))
  if (l2) {
    cpm.mat <- log2(cpm.mat + 1)
  }
  return(cpm.mat)
}

#' Performs a rank transformation on a given matrix.
#' 
#' @param dat.mat Matrix of data, usually gene expression (genes X samples).
#' @return Rank transformed matrix.
RankTransform <- function(dat.mat) {
  rank.mat <- apply(dat.mat, 2, rank)
  median <- apply(rank.mat, 1, median)
  mad <- apply(rank.mat, 1, mad)
  rank.mat <- (rank.mat - median) / mad
  return(rank.mat)
}

In [23]:
cpm.mat <- CPMTransform(raw.mat)
rank.mat <- RankTransform(cpm.mat)

ERROR: Error in t.default(dat.mat): argument is not a matrix


## Single table across cell types

In [66]:
mat.name <- paste0('../../../data/mouse_adrenal/preprocessed/snrna/', 'adrenal_Parse_10x_integrated_RNA.tsv')
subset.mat.name <- paste0('../../../data/mouse_adrenal/preprocessed/snrna/', 'adrenal_Parse_10x_integrated_RNA_subset.tsv')
sct.mat <- dat@assays$SCT@scale.data

In [67]:
dim(sct.mat)

[1]  3000 79209

In [68]:
sct.mat[1:6, 1:6]

,AAACAGCCAATATACC.ENCLB069PRB,AAACAGCCATGGCCCA.ENCLB069PRB,AAACATGCAAAGCTCC.ENCLB069PRB,AAACATGCAACTAGGG.ENCLB069PRB,AAACATGCAGCAATAA.ENCLB069PRB,AAACCAACAACAGCCT.ENCLB069PRB
Xkr4,-0.9224475,-0.7663493,-1.4341209,-0.7056428,-0.5758200,0.21506904
Sox17,-0.1967594,-0.2792488,0.3138141,-0.3808692,-0.4849125,0.02574090
St18,-1.0797149,-0.8436292,-1.9654508,-0.7221981,-0.5118624,-1.33412246
Sntg1,-0.6911699,-0.5918302,-1.0516712,-0.5367270,-0.4428372,-0.81139049
2610203C22Rik,-0.2887334,7.0255214,0.1545874,-0.4444054,4.2507128,-0.08726803
Prex2,-0.8133880,0.4206808,0.7182782,2.0115627,-1.6736534,1.74559885


In [70]:
ARACNeTable(sct.mat, mat.name, subset = FALSE)

## Cell type specific networks

---

# Scratch